## Lineáris és polinomiális regresszió a tök árának meghatározásához - 3. lecke
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika: Dasani Madipalli</figcaption>


<!--![Infografika: Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->

#### Bevezetés

Eddig megismerkedtél azzal, hogy mi is az a regresszió, például a tök árának meghatározásához használt adatokkal, amelyeket ebben a leckében is használni fogunk. Emellett vizualizáltad is az adatokat a `ggplot2` segítségével. 💪

Most készen állsz arra, hogy mélyebben belemerülj a gépi tanulás regressziójába. Ebben a leckében többet fogsz megtudni két regressziótípusról: *egyszerű lineáris regresszió* és *polinomiális regresszió*, valamint az ezek mögött álló matematikai alapokról.

> Ebben a tananyagban minimális matematikai előismeretet feltételezünk, és arra törekszünk, hogy a tanulók számára más területekről érkezve is érthető legyen. Figyelj a jegyzetekre, 🧮 kiemelésekre, diagramokra és egyéb tanulást segítő eszközökre, amelyek megkönnyítik a megértést.

#### Előkészületek

Emlékeztetőül: az adatokat azért töltöd be, hogy kérdéseket tegyél fel velük kapcsolatban.

- Mikor érdemes leginkább tököt vásárolni?

- Milyen árat várhatok egy mini tökös dobozért?

- Érdemes fél-bushel kosárban vagy 1 1/9 bushel dobozban vásárolni őket? Nézzük meg mélyebben az adatokat.

Az előző leckében létrehoztál egy `tibble`-t (a data frame modern újragondolása), és feltöltötted az eredeti adatállomány egy részével, standardizálva az árakat bushelre. Ezzel azonban csak körülbelül 400 adatpontot tudtál összegyűjteni, és csak az őszi hónapokra vonatkozóan. Talán több részletet is megtudhatunk az adatok természetéről, ha jobban megtisztítjuk őket? Majd meglátjuk... 🕵️‍♀️

Ehhez a feladathoz az alábbi csomagokra lesz szükségünk:

- `tidyverse`: A [tidyverse](https://www.tidyverse.org/) egy [R csomaggyűjtemény](https://www.tidyverse.org/packages), amely gyorsabbá, könnyebbé és szórakoztatóbbá teszi az adatfeldolgozást!

- `tidymodels`: A [tidymodels](https://www.tidymodels.org/) keretrendszer egy [csomaggyűjtemény](https://www.tidymodels.org/packages/) modellezéshez és gépi tanuláshoz.

- `janitor`: A [janitor csomag](https://github.com/sfirke/janitor) egyszerű eszközöket kínál a piszkos adatok vizsgálatához és tisztításához.

- `corrplot`: A [corrplot csomag](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) vizuális eszközt biztosít a korrelációs mátrix feltárásához, amely támogatja az automatikus változórendezést, hogy segítsen rejtett minták felismerésében a változók között.

A csomagok telepítése az alábbi módon történhet:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Az alábbi szkript ellenőrzi, hogy megvannak-e a szükséges csomagok a modul elvégzéséhez, és telepíti őket, ha hiányoznak.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Ezeket a fantasztikus csomagokat később betöltjük, és elérhetővé tesszük a jelenlegi R munkamenetünkben. (Ez csak illusztráció, a `pacman::p_load()` már elvégezte ezt helyettünk.)

## 1. Egy lineáris regressziós egyenes

Ahogy az 1. leckében tanultad, a lineáris regresszió célja, hogy képesek legyünk megrajzolni egy *legjobban illeszkedő egyenest*, amely:

-   **Megmutatja a változók közötti kapcsolatot**. Bemutatja a változók közötti összefüggést.

-   **Előrejelzéseket készít**. Pontos előrejelzéseket ad arra vonatkozóan, hogy egy új adatpont hol helyezkedne el az egyeneshez viszonyítva.

Ennek az egyenesnek a megrajzolásához egy statisztikai technikát használunk, amelyet **Legkisebb négyzetek módszerének** nevezünk. A `legkisebb négyzetek` kifejezés azt jelenti, hogy az egyenest körülvevő összes adatpontot négyzetre emeljük, majd összeadjuk. Ideális esetben ez az összeg a lehető legkisebb, mivel alacsony hibaszámot szeretnénk, vagyis `legkisebb négyzeteket`. Ezért a legjobban illeszkedő egyenes az, amely a legkisebb értéket adja a négyzetes hibák összegére - innen ered a *legkisebb négyzetek regresszió* elnevezés.

Ezt azért tesszük, mert olyan egyenest szeretnénk modellezni, amelynek a legkisebb kumulatív távolsága van az összes adatpontunktól. Az értékeket négyzetre emeljük, mielőtt összeadnánk őket, mivel az irány helyett az érték nagysága érdekel minket.

> **🧮 Mutasd a matekot**
>
> Ez az egyenes, amelyet *legjobban illeszkedő egyenesnek* nevezünk, [egy egyenlettel](https://en.wikipedia.org/wiki/Simple_linear_regression) fejezhető ki:
>
>     Y = a + bX
>
> `X` az '`magyarázó változó` vagy `prediktor`'. `Y` a '`függő változó` vagy `eredmény`'. Az egyenes meredeksége `b`, és `a` az y-metszet, amely arra utal, hogy `Y` értéke mennyi, amikor `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "meredekség = $y/x$")
    Infografika: Jen Looper
>
> Először számítsd ki a meredekséget, `b`.
>
> Más szavakkal, és utalva a tökadatok eredeti kérdésére: "jósoljuk meg a tök árát hónaponként", `X` az árra utal, míg `Y` az eladás hónapjára.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Infografika: Jen Looper
> 
> Számítsd ki `Y` értékét. Ha körülbelül 4 dollárt fizetsz, akkor április van!
>
> Az egyenest számító matematikának meg kell mutatnia az egyenes meredekségét, amely az y-metszettől is függ, vagyis attól, hogy `Y` hol helyezkedik el, amikor `X = 0`.
>
> Megfigyelheted ezeknek az értékeknek a számítási módját a [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html) weboldalon. Látogasd meg [ezt a legkisebb négyzetek kalkulátort](https://www.mathsisfun.com/data/least-squares-calculator.html), hogy megnézd, hogyan befolyásolják a számok értékei az egyenest.

Nem is olyan ijesztő, igaz? 🤓

#### Korreláció

Egy másik fontos fogalom, amit meg kell érteni, az **Korrelációs Együttható** az adott X és Y változók között. Egy szórásdiagram segítségével gyorsan vizualizálhatod ezt az együtthatót. Ha a pontok szépen egy egyenes mentén helyezkednek el, akkor magas korrelációról beszélünk, míg ha a pontok szanaszét helyezkednek el X és Y között, akkor alacsony korrelációról van szó.

Egy jó lineáris regressziós modell olyan lesz, amely magas (1-hez közelebb álló, mint 0-hoz) Korrelációs Együtthatóval rendelkezik a Legkisebb Négyzetek módszerével és egy regressziós egyenessel.


## **2. Tánc az adatokkal: adatkeret létrehozása modellezéshez**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Műalkotás: @allison_horst</figcaption>


<!--![Műalkotás: \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Töltsd be a szükséges könyvtárakat és az adatállományt. Alakítsd az adatokat egy adatkeretté, amely az adatok egy részhalmazát tartalmazza:

-   Csak azokat a tököket válaszd ki, amelyeket véka szerint áraznak

-   Alakítsd át a dátumot hónappá

-   Számítsd ki az árat a magas és alacsony árak átlagaként

-   Alakítsd át az árat, hogy tükrözze a véka szerinti mennyiségi árazást

> Ezeket a lépéseket a [korábbi leckében](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb) tárgyaltuk.


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

A puszta kaland szellemében fedezzük fel a [`janitor csomagot`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor), amely egyszerű funkciókat kínál a piszkos adatok vizsgálatához és tisztításához. Például nézzük meg az adataink oszlopneveit:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Jobban is csinálhatjuk. Tegyük ezeket az oszlopneveket `friendR`-ré azzal, hogy átalakítjuk őket a [snake_case](https://en.wikipedia.org/wiki/Snake_case) konvencióra a `janitor::clean_names` használatával. Ha többet szeretnél megtudni erről a függvényről: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Sokkal rendezettebb 🧹! Most egy tánc az adatokkal a `dplyr` segítségével, ahogy az előző leckében! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Szép munka!👌 Most már van egy tiszta, rendezett adathalmazod, amelyre építheted az új regressziós modelledet!

Egy szórási diagramot?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Egy szórási diagram emlékeztet minket arra, hogy csak augusztustól decemberig vannak havi adataink. Valószínűleg több adatra van szükségünk ahhoz, hogy lineáris módon tudjunk következtetéseket levonni.

Nézzük meg újra a modellezési adatainkat:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Mi lenne, ha meg szeretnénk jósolni egy tök `árát` a `város` vagy `csomag` oszlopok alapján, amelyek karakter típusúak? Vagy még egyszerűbben, hogyan találhatnánk meg a korrelációt (amihez mindkét bemenetnek numerikusnak kell lennie) például a `csomag` és az `ár` között? 🤷🤷

A gépi tanulási modellek legjobban numerikus jellemzőkkel működnek, nem pedig szöveges értékekkel, ezért általában szükséges a kategóriális jellemzőket numerikus reprezentációkká alakítani.

Ez azt jelenti, hogy meg kell találnunk egy módot arra, hogy átalakítsuk az előrejelzőinket, hogy a modellek hatékonyabban tudják használni őket. Ezt a folyamatot `jellemzők mérnöki tervezésének` nevezzük.


## 3. Adatok előfeldolgozása modellezéshez receptekkel 👩‍🍳👨‍🍳

Azokat a tevékenységeket, amelyek átalakítják a prediktor értékeket, hogy a modell hatékonyabban tudja használni őket, `jellemzők mérnöki tervezésének` nevezzük.

Különböző modelleknek eltérő előfeldolgozási követelményeik vannak. Például a legkisebb négyzetek módszere megköveteli a `kategóriális változók kódolását`, mint például hónap, fajta és városnév. Ez egyszerűen azt jelenti, hogy egy `kategóriális értékeket` tartalmazó oszlopot egy vagy több `numerikus oszlopra` fordítunk le, amelyek az eredeti oszlop helyét veszik át.

Például, tegyük fel, hogy az adataid tartalmazzák a következő kategóriális jellemzőt:

|  város   |
|:--------:|
| Denver   |
| Nairobi  |
|  Tokió   |

Alkalmazhatod az *ordinális kódolást*, hogy minden kategóriát egyedi egész értékkel helyettesíts, így:

| város |
|:-----:|
|   0   |
|   1   |
|   2   |

Pontosan ezt fogjuk tenni az adatainkkal!

Ebben a részben egy másik fantasztikus Tidymodels csomagot fogunk felfedezni: [recipes](https://tidymodels.github.io/recipes/) - amelyet arra terveztek, hogy segítsen az adatok előfeldolgozásában **mielőtt** a modellt betanítanánk. Lényegében egy recept egy olyan objektum, amely meghatározza, milyen lépéseket kell alkalmazni egy adathalmazon, hogy az készen álljon a modellezésre.

Most hozzunk létre egy receptet, amely előkészíti az adatainkat a modellezéshez úgy, hogy minden megfigyelést egyedi egész számmal helyettesít a prediktor oszlopokban:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Szuper! 👏 Most hoztuk létre az első receptünket, amely meghatározza az eredményt (árat) és a hozzá tartozó prediktorokat, valamint azt, hogy az összes prediktor oszlopot egész számokká kell kódolni 🙌! Nézzük meg gyorsan, hogyan is működik ez:

-   A `recipe()` függvény hívása egy formulával megadja a receptnek a változók *szerepeit*, a `new_pumpkins` adatok alapján. Például a `price` oszlop az `outcome` (eredmény) szerepet kapta, míg a többi oszlop a `predictor` (prediktor) szerepet.

-   A `step_integer(all_predictors(), zero_based = TRUE)` meghatározza, hogy az összes prediktort egy egész számokból álló halmazzá kell átalakítani, ahol a számozás 0-tól kezdődik.

Biztosan eszedbe jutott már valami ilyesmi: "Ez nagyon menő!! De mi van, ha meg kellene győződnöm arról, hogy a receptek pontosan azt csinálják, amit várok tőlük? 🤔"

Ez egy remek gondolat! Látod, ha egyszer meghatároztad a receptet, akkor becsülheted az adatok előfeldolgozásához szükséges paramétereket, majd kinyerheted a feldolgozott adatokat. Általában erre nincs szükség, amikor a Tidymodels-t használod (mindjárt látni fogjuk a szokásos megközelítést -\> `workflows`), de hasznos lehet, ha valamilyen ellenőrzést szeretnél végezni annak érdekében, hogy megbizonyosodj arról, hogy a receptek azt csinálják, amit elvársz.

Ehhez két további igére lesz szükséged: `prep()` és `bake()`. És ahogy mindig, kis R barátaink [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) által segítenek abban, hogy ezt jobban megértsd!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Illusztráció: @allison_horst</figcaption>


<!--![Illusztráció: \@allison_horst](../../../../../../2-Regression/3-Linear/images/recipes.png){width="550"}-->


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): becsli a szükséges paramétereket egy tanuló adathalmazból, amelyeket később más adathalmazokra is alkalmazni lehet. Például, egy adott prediktor oszlop esetében melyik megfigyelés kapja a 0, 1, 2 stb. egész számot.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): egy előkészített receptet vesz, és az abban szereplő műveleteket alkalmazza bármely adathalmazra.

Ezzel együtt, készítsük elő és alkalmazzuk a receptjeinket, hogy valóban megerősítsük, hogy a háttérben a prediktor oszlopok először kódolva lesznek, mielőtt a modell illesztésre kerül.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Hurrá! 🥳 A feldolgozott `baked_pumpkins` adatok összes prediktora kódolva van, ami megerősíti, hogy az előfeldolgozási lépések, amelyeket receptként definiáltunk, valóban a vártnak megfelelően működnek. Ez nehezebbé teszi az olvasást számodra, de sokkal érthetőbbé teszi a Tidymodels számára! Szánj egy kis időt arra, hogy kiderítsd, melyik megfigyelés lett egy megfelelő egész számra leképezve.

Érdemes megemlíteni, hogy a `baked_pumpkins` egy adatkeret, amelyen számításokat végezhetünk.

Például próbáljunk meg jó korrelációt találni az adataid két pontja között, hogy esetlegesen egy jó prediktív modellt építhessünk. Ehhez a `cor()` függvényt fogjuk használni. Írd be, hogy `?cor()`, hogy többet megtudj a függvényről.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Mint kiderült, csak gyenge összefüggés van a Város és az Ár között. Viszont valamivel jobb összefüggés mutatkozik a Csomag és annak Ára között. Ez logikus, nem? Általában minél nagyobb a termékdoboz, annál magasabb az ár.

Amíg itt tartunk, próbáljuk meg vizualizálni az összes oszlop korrelációs mátrixát a `corrplot` csomag segítségével.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Sokkal jobb.

Egy jó kérdés, amit most feltehetünk az adatokkal kapcsolatban: '`Milyen árat várhatok egy adott tökcsomagra?`' Vágjunk is bele!

> Megjegyzés: Amikor a **`bake()`** függvényt használod az előkészített recepttel (**`pumpkins_prep`**) és **`new_data = NULL`** értékkel, akkor a feldolgozott (azaz kódolt) tanulóadatokat kapod meg. Ha lenne egy másik adathalmazod, például egy tesztkészlet, és szeretnéd látni, hogyan dolgozná fel azt a recept, egyszerűen süsd meg (**`bake`**) a **`pumpkins_prep`** receptet a **`new_data = test_set`** paraméterrel.

## 4. Készítsünk egy lineáris regressziós modellt

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika: Dasani Madipalli</figcaption>


<!--![Infografika: Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Most, hogy elkészítettük a receptet, és ténylegesen megerősítettük, hogy az adatok megfelelően elő lesznek dolgozva, építsünk egy regressziós modellt, hogy megválaszoljuk a kérdést: `Milyen árat várhatok egy adott tökcsomag esetében?`

#### Tanítsunk egy lineáris regressziós modellt a tanuló adathalmazon

Ahogy valószínűleg már kitaláltad, a *price* oszlop az `eredmény` változó, míg a *package* oszlop a `prediktor` változó.

Ehhez először szétosztjuk az adatokat úgy, hogy 80% a tanuló adathalmazba, 20% pedig a teszt adathalmazba kerüljön, majd definiálunk egy receptet, amely a prediktor oszlopot egész számokká kódolja, és létrehozunk egy modell specifikációt. Nem fogjuk előkészíteni és sütni a receptet, mivel már tudjuk, hogy az adatok előfeldolgozása a vártnak megfelelően fog történni.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Szép munka! Most, hogy van egy receptünk és egy modell specifikációnk, meg kell találnunk a módját, hogy ezeket egy olyan objektumba csomagoljuk, amely először előkészíti az adatokat (a háttérben előkészít és süt), illeszti a modellt az előkészített adatokra, és lehetőséget biztosít az esetleges utófeldolgozási tevékenységekre is. Na, ez aztán megnyugtató, nem igaz? 🤩

A Tidymodels-ben ezt a praktikus objektumot [`workflow`](https://workflows.tidymodels.org/) néven ismerjük, és kényelmesen tartalmazza a modellezési komponenseidet! Ezt hívnánk *pipelines*-nak *Python*-ban.

Szóval csomagoljunk össze mindent egy workflow-ba! 📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

Ráadásul egy munkafolyamatot nagyjából ugyanúgy lehet illeszteni/betanítani, mint egy modellt.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

A modell tanítása során kapott eredményekből láthatjuk a tanulás során meghatározott együtthatókat. Ezek az együtthatók a legjobban illeszkedő egyenes együtthatói, amely a legkisebb összesített hibát adja az aktuális és a becsült változó között.

#### A modell teljesítményének értékelése a tesztkészlettel

Itt az idő, hogy megnézzük, hogyan teljesített a modell 📏! Hogyan tegyük ezt?

Most, hogy betanítottuk a modellt, használhatjuk a `parsnip::predict()` függvényt, hogy előrejelzéseket készítsünk a tesztkészletre. Ezután összehasonlíthatjuk ezeket az előrejelzéseket a tényleges címkeértékekkel, hogy értékeljük, mennyire jól (vagy éppen nem!) működik a modell.

Kezdjük azzal, hogy előrejelzéseket készítünk a tesztkészletre, majd összekapcsoljuk az oszlopokat a tesztkészlettel.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Igen, éppen most tanítottál be egy modellt, és használtad előrejelzések készítésére!🔮 Mennyire jó? Nézzük meg a modell teljesítményét!

A Tidymodels-ben ezt a `yardstick::metrics()` segítségével végezzük! Lineáris regresszió esetén az alábbi metrikákra összpontosítunk:

-   `Root Mean Square Error (RMSE)`: Az [MSE](https://en.wikipedia.org/wiki/Mean_squared_error) négyzetgyöke. Ez egy abszolút metrikát ad ugyanabban az egységben, mint a címke (ebben az esetben egy tök ára). Minél kisebb az érték, annál jobb a modell (egyszerűen fogalmazva, ez az átlagos ár, amellyel az előrejelzések tévesek!).

-   `Coefficient of Determination (általában R-négyzet vagy R2 néven ismert)`: Egy relatív metrika, amelynél minél magasabb az érték, annál jobb a modell illeszkedése. Lényegében ez a metrika azt mutatja meg, hogy a modell mennyire képes magyarázni az előrejelzett és a valós címkeértékek közötti varianciát.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Ott megy a modell teljesítménye. Nézzük meg, hogy jobb képet kapunk-e, ha vizualizálunk egy szórásdiagramot a csomagokról és az árakról, majd az előrejelzéseket felhasználva ráillesztünk egy legjobban illeszkedő egyenest.

Ez azt jelenti, hogy elő kell készítenünk és feldolgoznunk kell a tesztkészletet, hogy kódoljuk a csomag oszlopot, majd ezt összekapcsoljuk a modellünk által készített előrejelzésekkel.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Nagyszerű! Ahogy látható, a lineáris regressziós modell nem igazán általánosítja jól a csomag és annak megfelelő ára közötti kapcsolatot.

🎃 Gratulálok, éppen létrehoztál egy modellt, amely segíthet néhány tökféle árának előrejelzésében. Az ünnepi tökös kerted gyönyörű lesz. De valószínűleg létrehozhatsz egy jobb modellt is!

## 5. Készíts egy polinomiális regressziós modellt

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika: Dasani Madipalli</figcaption>


<!--![Infografika: Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Néha az adataink nem mutatnak lineáris összefüggést, de mégis szeretnénk előre jelezni egy eredményt. A polinomiális regresszió segíthet bonyolultabb, nemlineáris kapcsolatok esetén is előrejelzéseket készíteni.

Vegyük például a tökök adatállományában a csomagolás és az ár közötti kapcsolatot. Bár néha van lineáris összefüggés a változók között – például minél nagyobb a tök térfogata, annál magasabb az ára –, előfordulhat, hogy ezek a kapcsolatok nem ábrázolhatók síkként vagy egyenes vonalként.

> ✅ Itt van [néhány további példa](https://online.stat.psu.edu/stat501/lesson/9/9.8) olyan adatokra, amelyekhez polinomiális regresszió alkalmazható
>
> Nézd meg újra a korábbi ábrán a Fajta és Ár közötti kapcsolatot. Úgy tűnik, hogy ezt a szórásdiagramot feltétlenül egy egyenes vonallal kellene elemezni? Talán nem. Ebben az esetben érdemes lehet kipróbálni a polinomiális regressziót.
>
> ✅ A polinomok olyan matematikai kifejezések, amelyek egy vagy több változóból és együtthatóból állhatnak

#### Polinomiális regressziós modell betanítása a tanuló adathalmazon

A polinomiális regresszió egy *görbe vonalat* hoz létre, amely jobban illeszkedik a nemlineáris adatokhoz.

Nézzük meg, hogy egy polinomiális modell jobban teljesít-e az előrejelzések készítésében. Hasonló eljárást követünk, mint korábban:

-   Hozzunk létre egy receptet, amely meghatározza azokat az előfeldolgozási lépéseket, amelyeket az adatok modellezésre való előkészítése során el kell végezni, például: prediktorok kódolása és *n* fokú polinomok kiszámítása

-   Készítsünk egy modell specifikációt

-   Csomagoljuk össze a receptet és a modell specifikációt egy munkafolyamatba

-   Hozzunk létre egy modellt a munkafolyamat illesztésével

-   Értékeljük, hogy a modell mennyire teljesít jól a tesztadatokon

Vágjunk is bele!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Értékeljük a modell teljesítményét

👏👏Létrehoztál egy polinomiális modellt, nézzük meg, hogyan teljesít az előrejelzések során a tesztkészleten!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Hurrá, értékeljük, hogyan teljesített a modell a test_set-en a `yardstick::metrics()` segítségével.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Sokkal jobb teljesítmény.

Az `rmse` körülbelül 7-ről körülbelül 3-ra csökkent, ami azt jelzi, hogy csökkent a tényleges ár és az előrejelzett ár közötti hiba. Ezt *nagyjából* úgy értelmezhetjük, hogy átlagosan a hibás előrejelzések körülbelül 3 dollárral tévednek. Az `rsq` körülbelül 0,4-ről 0,8-ra nőtt.

Mindezek a mutatók azt jelzik, hogy a polinomiális modell sokkal jobban teljesít, mint a lineáris modell. Szép munka!

Nézzük meg, hogy tudjuk-e ezt vizualizálni!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Láthatod, hogy egy görbe vonal jobban illeszkedik az adataidhoz! 🤩

Ezt még simábbá teheted, ha egy polinomiális képletet adsz át a `geom_smooth`-nak, például így:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Akárcsak egy sima görbe!🤩

Így készíthetsz egy új előrejelzést:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


A `polinomiális modell` előrejelzése logikusnak tűnik, tekintve a `price` és `package` szórásdiagramjait! És ha ez egy jobb modell, mint az előző, ugyanazon adatok alapján, akkor számolnod kell ezekkel a drágább tökökkel!

🏆 Szép munka! Egy leckében két regressziós modellt hoztál létre. A regresszióval kapcsolatos utolsó részben a logisztikus regresszióról fogsz tanulni, amely kategóriák meghatározására szolgál.

## **🚀Kihívás**

Tesztelj több különböző változót ebben a notebookban, hogy lásd, hogyan függ össze a korreláció a modell pontosságával.

## [**Előadás utáni kvíz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Áttekintés és önálló tanulás**

Ebben a leckében a lineáris regresszióról tanultunk. Vannak más fontos regressziós technikák is. Olvass utána a Stepwise, Ridge, Lasso és Elasticnet módszereknek. Egy jó kurzus, amely segít többet tanulni, a [Stanford Statistical Learning kurzus](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Ha többet szeretnél megtudni a fantasztikus Tidymodels keretrendszer használatáról, nézd meg az alábbi forrásokat:

-   Tidymodels weboldal: [Kezdd el a Tidymodels használatát](https://www.tidymodels.org/start/)

-   Max Kuhn és Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **KÖSZÖNET ILLETI:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) az elképesztő illusztrációkért, amelyek barátságosabbá és vonzóbbá teszik az R-t. További illusztrációkat találhatsz a [galériájában](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Felelősség kizárása**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Fontos információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
